In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

from pprint import pprint

#### Задание 1.
##### Обязательная часть

Будем парсить страницу со свежими новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

##### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [2]:
HABR_URL = 'https://habr.com/ru/all/'
KEYWORDS = ['python', 'парсинг', 'Python', 'Парсинг', 'JavaScript', 'парсер']
DATA_FRAME = {"date": [], "title": [], "link": [], "text": []}

In [3]:
req = requests.get(HABR_URL)
soup = BeautifulSoup(req.text, 'html.parser')
# извлекаем посты
posts = soup.find_all('article', class_='post')

In [4]:
def get_post_text(link):
    req = requests.get(link)
    soup = BeautifulSoup(req.text, 'html.parser')
    text = soup.find('div', {'id': 'post-content-body'}).get_text()
    return text
        
def filter_posts(post):
    full_link = post.find('a', class_='post__title_link')
    link = full_link.get('href')
    title = full_link.get_text()
    date = post.find('span', class_='post__time')
    text = get_post_text(link)
    
    if any([word in text for word in KEYWORDS]):
        DATA_FRAME["link"].append(link)
        DATA_FRAME["title"].append(title)
        DATA_FRAME["date"].append(date)
        DATA_FRAME["text"].append(text)
        
for post in posts:
    filter_posts(post)
        
post_df = pd.DataFrame(DATA_FRAME)

post_df

,date,title,link,text
0,[сегодня в 13:41],Калькулятор Wolframalpha в диалоге Telegram,https://habr.com/ru/post/515146/,Идея\r\nВ диалогах телеграма я очень часто исп...
1,[сегодня в 11:52],"Создание голосового ассистента на Python, часть 1",https://habr.com/ru/post/515128/,"Добрый день. Наверное, все смотрели фильмы про..."
2,[сегодня в 11:50],5 стратегий успешного поиска работы в сфере IT,https://habr.com/ru/post/515126/,\n\r\nПандемия COVID-19 успела внести свои кор...
3,[сегодня в 11:16],Blender для (геофизического) моделирования и в...,https://habr.com/ru/post/515122/,"Недавно мы обсудили, как дополнить ранее постр..."
4,[сегодня в 11:00],Пишем интерпретатор BASIC в стиле 80-х,https://habr.com/ru/company/vdsina/blog/515032/,\n\r\nВ течение нескольких лет я работала над ...


#### Задание 2.
##### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.


In [5]:
EMAIL = ["ekaterina.frolova@gmail.com", "maksim.petrov@mail.ru", "ivanivanov@gmail.com"]
URL = "https://digibody.avast.com/v1/web/leaks"

DATA = {"email": [], "leak_date": [], "leak_source": [], "description": []}

def check_email(email):
    params = {"email": email}
    resp = requests.post(URL, json=params)
    result = resp.json()["value"]
    if result:
        for leak in result:
            DATA["email"].append(email)
            DATA["leak_date"].append(leak["leak_date"])
            DATA["leak_source"].append(leak["domain"])
            DATA["description"].append(leak["leak_info"]["description"])
    else:
        DATA["email"].append(email)
        DATA["leak_date"].append("")
        DATA["leak_source"].append("")
        DATA["description"].append("")
    
for email in EMAIL:
    check_email(email)
    
leaks_df = pd.DataFrame(DATA)

leaks_df

,email,leak_date,leak_source,description
0,ekaterina.frolova@gmail.com,,,
1,maksim.petrov@mail.ru,1513900800000,,The proliferation of stolen or leaked database...
2,maksim.petrov@mail.ru,1549411200000,,"On January 7, 2019, an online user named Sanix..."
3,maksim.petrov@mail.ru,1507507200000,,The proliferation of stolen or leaked database...
4,maksim.petrov@mail.ru,1590624000000,stalker.so,"In January 2020, the Russian multiplayer video..."
5,maksim.petrov@mail.ru,1487030400000,mail.ru,"In July and August 2016, two criminals execute..."
6,maksim.petrov@mail.ru,1488931200000,,This source has been marked as sensitive due t...
7,maksim.petrov@mail.ru,1549411200000,,"On January 7, 2019, an online user named Sanix..."
8,maksim.petrov@mail.ru,1549411200000,,"On January 7, 2019, an online user named Sanix..."
9,maksim.petrov@mail.ru,1548720000000,,"On January 7, 2019, an online user named Sanix..."


##### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [6]:
TOKEN = "введите свой токен"

URL = "https://api.vk.com/method/" 
GET_GROUP_METHOD = "groups.getById"
GET_POSTS_METHOD = "wall.get"
TARGET = "netology"
V = "5.21"

main_params = {
    "access_token": TOKEN,
    "v": V
}

get_group_params = {
    "group_id": TARGET,
}

get_group_params.update(main_params)
group = requests.get(URL+GET_GROUP_METHOD, get_group_params)
group_id = group.json()["response"][0]["id"]

get_posts_params = {
    "owner_id": -group_id,
    "count": 50,
}

get_posts_params.update(main_params)
posts = requests.get(URL+GET_POSTS_METHOD, get_posts_params).json()["response"]["items"]

posts_df = {"post_date": [], "post_text": []}

for post in posts:
    posts_df["post_date"].append(post["date"])
    posts_df["post_text"].append(post["text"])
    
posts_df = pd.DataFrame(posts_df)
posts_df

,post_date,post_text
0,1597402808,Всем привет! Сегодня первый день карьерного кв...
1,1597402812,А вот и первое задание карьерного квеста. Мене...
2,1597390920,"Больше, ещё больше знаний🤓 Разбираемся с дизай..."
3,1597331640,"«Нужно работать так, словно таланта у тебя воо..."
4,1597304520,"Составляем карту пути клиента, открываем калит..."
5,1597246920,Вечер среды — самое время для поиска формулы б...
6,1597218120,Синдром самозванца — склонность преуменьшать с...
7,1597156920,"Несколько советов, которые помогут держать зад..."
8,1597130760,Закройте окошко — сквозит 🤧 Откройте отчёт — э...
9,1597069560,"Фотку котика, загадочный пейзаж или всё-таки с..."
